# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901996


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<01:06,  2.30s/it]

Rendering models:  10%|▉         | 3/31 [00:04<00:47,  1.69s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:55,  2.07s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:41,  1.59s/it]

...

Rendering models:  90%|█████████ | 28/31 [00:16<00:01,  2.57it/s]

Rendering models:  94%|█████████▎| 29/31 [00:16<00:00,  2.62it/s]

Rendering models:  97%|█████████▋| 30/31 [00:16<00:00,  2.94it/s]

Rendering models: 100%|██████████| 31/31 [00:17<00:00,  2.86it/s]

equidad1                               0.000666
marius_aga                             0.002955
haskielr                               0.002442
Melina_t                               0.000678
not-logged-in-1d1ba37cbc6d34025018     0.004094
tingard                                0.000694
michiveda                              0.002502
start056                               0.001411
EKFritcher                             0.001017
not-logged-in-545b59d533d78a4f8dea     0.000820
not-logged-in-61a0765eeab9a5074a2a     0.003678
not-logged-in-2bc079ac48efb4acdd24     0.006888
MilesGray                              0.003508
not-logged-in-ba8996c2441118f8e60e     0.001734
pangeli5                               0.000996
acapirala                              0.014298
Lavadude                               0.003014
lildipper_246                         18.007123
TateNachampassak                       0.001248
gmann2023                              0.008132
slambier                               0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())